In [1]:
## 1. Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

In [2]:
# Optional display settings
pd.set_option('display.max_columns', None)

In [3]:
## 2. Load Dataset
df = pd.read_csv("Leads.csv")
print("Initial shape of dataset:", df.shape)
df.head()

Initial shape of dataset: (9240, 37)


,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,NaN,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Interested in other courses,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,NaN,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


In [4]:
## 3. Data Cleaning
# Drop identifier columns
df.drop(['Prospect ID', 'Lead Number'], axis=1, inplace=True)

In [5]:
# Replace 'Select' with NaN
df.replace('Select', np.nan, inplace=True)

In [6]:
# Drop columns with more than 30% missing values
threshold = len(df) * 0.7
df = df.dropna(thresh=threshold, axis=1)

In [7]:
# Drop remaining rows with missing values
df.dropna(inplace=True)

In [8]:
print("Shape after cleaning:", df.shape)

Shape after cleaning: (4925, 25)


In [9]:
df.head()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Update me on Supply Chain Content,Get updates on DM Content,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
1,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,No,No,No,No,Email Opened
2,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Student,Better Career Prospects,No,No,No,No,No,No,No,No,No,No,No,Yes,Email Opened
3,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,No,No,No,No,Modified
4,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,No,No,No,No,Modified
6,Landing Page Submission,Google,No,No,1,2.0,1640,2.0,Email Opened,India,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,No,No,No,No,Modified


In [10]:
## 4. Encode Categorical Variables
df_encoded = pd.get_dummies(df, drop_first=True)
print("Shape after encoding:", df_encoded.shape)

Shape after encoding: (4925, 94)


In [11]:
## 5. Train-Test Split
X = df_encoded.drop("Converted", axis=1)
y = df_encoded["Converted"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
## 6. Train the Logistic Regression Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

C:\Users\ayanm\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [13]:
## 7. Evaluate the Model
y_pred = model.predict(X_test)
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy Score: 0.7786802030456853
Confusion Matrix:
 [[459  96]
 [122 308]]
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.83      0.81       555
           1       0.76      0.72      0.74       430

    accuracy                           0.78       985
   macro avg       0.78      0.77      0.77       985
weighted avg       0.78      0.78      0.78       985



In [14]:
## 8. Save the Trained Model and Columns
# Save the model
with open("lead_model.pkl", "wb") as file:
    pickle.dump(model, file)

# Save the model's input columns
with open("model_columns.pkl", "wb") as file:
    pickle.dump(list(X.columns), file)

print("Model and column structure saved successfully.")

Model and column structure saved successfully.


In [15]:
## 9. Predict on New Input Example
# Simulated input (must match encoded structure)
input_data = {
    'TotalVisits': 5,
    'Total Time Spent on Website': 10,
    'Page Views Per Visit': 2,
    'Lead Origin_Landing Page Submission': 1,
    'Lead Origin_Lead Add Form': 0,
    'Lead Source_Google': 0,
    'Lead Source_Welingak Website': 1,
    'Do Not Email_Yes': 0,
    'Do Not Call_Yes': 0,
    'Converted': 0  # This should be removed before prediction
}

In [17]:
# Create dataframe and align with model columns
input_df = pd.DataFrame([input_data])
if 'Converted' in input_df.columns:
    input_df.drop('Converted', axis=1, inplace=True)

for col in X.columns:
    if col not in input_df.columns:
        input_df[col] = 0

input_df = input_df[X.columns]  # reorder

In [18]:
# Load model and predict
with open("lead_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

prediction = loaded_model.predict(input_df)[0]
print("Predicted Conversion:", "Yes" if prediction == 1 else "No")

Predicted Conversion: Yes
